# Introduction

The project entails understanding the load shedding crisis that Eskom has in doing so one component would be to understand the customer sentiments. One way would be would be to track customer satisfaction around Eskom products and services.A data pipeline is created  that can stream twitter data into an existing database, which will later on be analysed for sentiment. 

**Why a Data Pipeline ?**

Data pipelines facilitate the flow of data from a data source to a destination, with the aim of making the data suitable for consumption by the destination system. They are typically made up of a sequence of processing nodes connected in series which apply operations to the data as it flows through them and are typically designed to be reusable and maintainable. Data pipelines can vary depending on the volume, frequency, or nature of data flowing through them.The data pipeline used in this project is an **ETL Pipeline** which is a data pipeline is broken down into data Extraction, data Transforming, and data Loading.



**Building The ETL Pipeline**

The ETL pipeline consists of the following:

1. Function to connect to twitter and scrapes "Eskom_SA" tweets.
<br>
<br>
2. Cleans/Processes the tweets from the scraped tweets which will create a dataframe with two new columns using the following functions: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a) Hashtag Remover from Analyse Functions
<br>
<br>
3. Functions which connects to your SQL database and uploads the tweets into the table you store the tweets in the database.

In [6]:
# General:
import tweepy           # To consume Twitter's API
import pandas as pd     # To handle data
import numpy as np      # For numerical computation
import json
# For plotting and visualization:
from IPython.display import display
import pyodbc


# Consumer and Access details

Fill in your Consumer and Access details you should have recieved when applying for a Twitter API. 

In [19]:
# Consumer:
CONSUMER_KEY    = " " 
CONSUMER_SECRET = " " 

# Access:
ACCESS_TOKEN  = " "
ACCESS_SECRET = " "

In [20]:
# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with access and consumer keys from Twitter.
    """

    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth, timeout=1000)
    return api

# Function 1:

The following function:
- Scrapes _"Eskom_SA"_ tweets from Twitter. 

Function Specifications:
- The function returns a dataframe with the scraped tweets with just the "_Tweets_" and "_Date_". 
- Will take in the ```consumer key,  consumer secret code, access token``` and ```access secret code```.

NOTE:
The dataframe has the same column names as those in your SQL Database table where you store the tweets.

In [21]:
def twitter_df(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET ):
    '''
    The function scrapes tweets  
    
    Parameters:
    -------------
    extractor : an extractor object that authenticates the user 
                using the Twitter API credentials 
    tweets : extract tweets from any Twitter account,the user_timeline raptures
             the tweets from the screen name(known as the twitter account name).   
    df: creates a dataframe that places the tweets and when they were created 
        into a tweets and date column                         


    Returns:
    -------------
    A dataframe of scraped tweets in a Tweets and Date column.

    '''
    extractor = twitter_setup()
    tweets = extractor.user_timeline(screen_name="Eskom_SA", 
                                 count=200,
                                include_rts=False)
    
    df = pd.DataFrame(data=np.column_stack([[tweet.text for tweet in tweets],
                                          [(tweet.created_at) for tweet in tweets]]),
                                          columns=['Tweets','Date'])
    
    return df

# Function 2: Removing hashtags and the municipalities

The following function:
- Uses a function that extracts the hashtags and municipalities into it's own column in a new data frame. 

Function Specifications:
- The function  takes in the pandas dataframe created in Function 1 and return a new pandas dataframe. 

In [22]:
twitter_df(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET )

Tweets                Date
0    Looking for alternative ways to keep you warm ... 2020-09-02 16:00:21
1    @Mkabayi11 @errandwowor Eskom does have an app... 2020-09-02 15:27:05
2    #EskomEasternCape advises customers of Alice o... 2020-09-02 14:29:42
3    Customers are urged to switch off all non-esse... 2020-09-02 13:15:01
4    @Mkabayi11 Eskom se Push is a private app. It ... 2020-09-02 12:52:03
..                                                 ...                 ...
186     @Rty273 Hi. Please DM us your contact details. 2020-08-22 15:16:05
187  DYK that you can use MyEskom Customer App to r... 2020-08-22 15:00:04
188  @maginyamulweli Pls, forward us your reference... 2020-08-22 14:37:03
189  Eskom Load Reduction Notice\n\nDate: 22 August... 2020-08-22 13:00:08
190  @just_f1f1 Please, log a call on MyEskom Custo... 2020-08-22 12:48:43

[191 rows x 2 columns]

In [23]:
def extract_municipality_hashtags(df):
    

    '''
    The function cleans scraped tweets by extracting the hashtags and 
    municipalities into their own column in a new dataframe.  
    
    Parameters:
    -------------
    mun_dict : a dictionary of municipality twitter handles 
               and municipality name 
    df : defines the scraped tweets df created in fucntion 
         1 as the input    
    df['municipality'] : Creates a municipality column with 
                         municipalities extrated from the tweets column                         
    df['hashtags'] : Creates a hashtags column with hashtags extrated 
                     from the tweets column                    
    df['Tweets'] : Cleans the Tweets by removing the hashtags and
                   municipalities in the  Tweets column

    Returns:
    -------------
    An updated dataframe that has a tweets column with removed 
    hashtags and municipalities,the municipalities and hashtags
    are extracted into their own columns.

    '''

    mun_dict = {'@CityofCTAlerts' : 'Cape Town',
                '@CityPowerJhb' : 'Johannesburg',
                '@eThekwiniM' : 'eThekwini' ,
                '@EMMInfo' : 'Ekurhuleni',
                '@centlecutility' : 'Mangaung',
                '@NMBmunicipality' : 'Nelson Mandela Bay',
                '@CityTshwane' : 'Tshwane'}
    
    handles = list(mun_dict.keys())
    
    df = twitter_df(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET )
    
    df['municipality'] = df['Tweets'].str.extract('({})'.format('|'.join(handles)),
                         expand=False).fillna(np.nan)
    df['municipality'] = df['municipality'].map(mun_dict)
    
    df['hashtags'] = df['Tweets'].str.findall(r'#.*?(?=\s|$)')
    df['hashtags'] = df['hashtags'].apply(lambda x: np.nan if len(x)==0 else [x.lower() for x in x])
    
    df['Tweets'] = df['Tweets'].str.replace(r'\B#\w*[a-zA-Z]+\w*', '')
    df['Tweets'] = df['Tweets'].str.replace('({})','')
    
    return df

# Function 3: Updating SQL Database with pyODBC

The following function:
- Connects and updates your SQL database. 

Function Specifications:
- The function should take in a pandas dataframe created in Function 2. 
- Connect to your SQL database.
- Update the table you store your tweets in.
- Not return any output.

In [27]:
def pyodbc_twitter(connection, df, twitter_table):
   
    '''
    The function connect to the SQL database and updates 
    the twitter_table using the dataframe from function 2. 


    Parameters:
    -----------
    df = dataframe from function 2
    connection = connects to the SQL Database
    query = defines the query which updates the SQL twitter_table. 

    Returns:
    ----------
    The function does not return an output.
    
    '''
    df = extract_municipality_hashtags(twitter_df)
 
    # host = COMPUTERNAME\SQLEXPRESS
    connection = pyodbc.connect(driver='{SQL Server}',
                          host='.\SQLEXPRESS', 
                          database='gather_eskom',
                          trusted_connection='tcon',
                          user='sa',
                          autocommit=True
                           )
    
    cursor = connection.cursor()     

    query = "INSERT INTO Tweets ([Tweet],[Date]) VALUES(?,?)"

    for index,row in df.iterrows():
        cursor.execute(query, (row[0],row[1]))

    connection.commit()
    cursor.close()
    connection.close()


    return None 

In [28]:
pyodbc_twitter('','', '')